In [1]:
import ast
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout

In [2]:
df = pd.read_csv('rnn_task_environments_2021-03-17.csv', header=0)

In [283]:
df = df[:1]
df

,ID,sequence,coordinates,start_coordinates,strong_k,mid_k,food_k,containment
0,a1,pocgkr,"p: (0.008034,0.957082,0.6890539999999999);o: (...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,"p,o",NaN,NaN


In [302]:
def create_input_data(dataset):
    data = []
    target = []
    
    for x in range(0, len(dataset)):
        for i in range(0, len(dataset['sequence'][x]) - 1):
            start_coords = ast.literal_eval(dataset['start_coordinates'][x])
            coords = {key: ast.literal_eval(value) for key, value in
                     (elem.split(': ') for elem in dataset.at[x, 'coordinates'].split(';'))}
            
            if i == 0:
                features = [np.asarray('0'),
                           np.array([coords]),
                           np.asarray(start_coords[i]),
                           np.asarray(dataset['strong_k'][x]),
                           np.asarray(dataset['mid_k'][x]),
                           np.asarray(dataset['food_k'][x]),
                           np.asarray(dataset['containment'][x])]
                
                next_action = dataset['sequence'][x][i+1]
                
                
                data.append(features)
                target.append(next_action)
            
            else:
                features = [np.asarray(dataset['sequence'][x][i]),
                           np.array([coords]),
                           np.asarray(start_coords[i]),
                           np.asarray(dataset['strong_k'][x]),
                           np.asarray(dataset['mid_k'][x]),
                           np.asarray(dataset['food_k'][x]),
                           np.asarray(dataset['containment'][x])]
                
                next_action = dataset['sequence'][x][i+1]
                
                data.append(features)
                target.append(next_action)
                
    data = np.asarray(data)
    data = np.expand_dims(data, -1)
    
    target = np.asarray(target)
    target = np.expand_dims(target, -1)
                
    return data, target

In [303]:
data, target = create_input_data(df)

In [304]:
data[0][0]

array([nan])

In [305]:
data.shape

(5, 2, 1)

In [306]:
target.shape

(5, 1)

In [307]:
def create_model(units):
    model = Sequential()
    
    model.add(GRU(units=units, return_sequences=True,
                 input_shape=[data.shape[1], data.shape[2]]))
    model.add(Dropout(0.2))
    
    # hidden layer
    model.add(GRU(units=units))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    
    # compile model
    model.compile(optimizer='adam', loss='mse')
    
    return model

In [308]:
gru = create_model(64)

In [309]:
def fit_model(model):
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    
    history = model.fit(data, target, epochs=100, validation_split=0.2,
                       batch_size=16, shuffle=False, callbacks=[early_stop])
    
    return history

In [310]:
history_gru = fit_model(gru)

Train on 4 samples, validate on 1 samples
Epoch 1/100
4/4 [==============================] - 1s 329ms/sample


UnimplementedError:  Cast string to float is not supported
	 [[node loss/dense_15_loss/Cast (defined at <ipython-input-309-260a33db2b56>:5) ]] [Op:__inference_distributed_function_37407]

Function call stack:
distributed_function
